In [ ]:
import initialization as init
import tokenization as token
import pprint as pp

def initialize():
    init.del_index()
    init.create_index()
    init.index_doc()

def print_ingredients(recipe):
    print("Ingredients:")
    for ingredient in  recipe['ingredients']:
        print("\t" + ingredient['displayText'])

def print_recipe(recipe):
    print("========== " + recipe['displayName'] +" ==========")
    print_ingredients(recipe)
    print("Recipe:")
    i = 1
    for instruction in recipe['instructions']:
        print("\t Step " + str(i) + ": " + instruction['stepText'])
        i += 1

def start_recipe(recipe):
    step = -1
    print("BOT: If you'd like to start the recipe say next, you can check the ingredients and if you'd like you can end the recipe.")
    while True:
        x = input(" USER: ")
        if x == "next":
            if  step <= len(recipe['instructions']):
                step += 1
            init.displayStep('Step ' + str(step) + ': ' + str(recipe['instructions'][step]['stepText']))
            print("BOT: If you want you can see the next step, can go to the previous step or you can end the recipe.")
        elif x == "previous":
            if step > 0:
                step -= 1
            init.displayStep('Step ' + str(step)+': ' +str(recipe['instructions'][step]['stepText']))
            print("BOT: If you want you can see the next step, can go to the previous step or you can end the recipe.")
        elif x == "ingredients":
            print_ingredients(recipe)
        elif x == "end":
            break
        else:
            print("BOT: I'm sorry, i couldn't understand the command. Commands: next, previous, ingredients and end.")
        
def search_recipe(query, withI, withoutI, searchEngine=None):
    if searchEngine == None:
        response = init.search_opensearch_conditions(query,withI,withoutI)
    else:
        if searchEngine == "opensearch":
            response = init.search_opensearch(query)
        elif searchEngine == "dual_encoders":
            response = init.search_dual_encoders(query)
        else:
            print("invalid search engine")
            exit()
            
    #pp.pprint(init.recipe_book[response['hits']['hits'][0]['fields']['id'][0]])    
    result_number = 0
    if not response['hits']['hits']:
        print("BOT: No relevant results found.")
        return "Fail"
            
   
    imgs = []
    txts = []
    serves = []
    more = True
    while more == True:
        more = False
        print("BOT: Great! These are the results I found:")
        for hit in response['hits']['hits'][result_number : result_number + 4 if len(response['hits']['hits']) >= 4 else len(response['hits']['hits'])]:
            id = str(response['hits']['hits'][result_number]['_source']['doc_id'])
            txts.append(hit['_source']['description'].split(' - ')[0])
            imgs.append(init.recipe_book[id]['images'][0]['url'])
            serves.append('Serves %s people' % str(init.recipe_book[id]['servings']) if str(init.recipe_book[id]['servings']) != 'None' else '')
            result_number += 1
        
        init.displayResults(txts[-4], imgs[-4], serves[-4],
                            txts[-3], imgs[-3], serves[-3],
                            txts[-2], imgs[-2], serves[-2],
                            txts[-1], imgs[-1], serves[-1])    
        prompt = "BOT: Select a number from 1 to " + str(result_number) + ", type 'more' to show more results or 'back' go back to searching: "   
        select = input(prompt)
    
        if select.isdigit() and int(select) <= result_number and int(select) >= 1:
            return init.recipe_book[str(response['hits']['hits'][int(select) - 1]['_source']['doc_id'])]
        elif select == "back":
            return "back"
        elif select == "more":
            if result_number >= len(response['hits']['hits']):
                print ("BOT: No more relevant results.")
                more = True
            else:
                more = True
        else:
            print("Invalid input")
            if result_number >= 4:
                result_number -= 4
            else:
                result_number = 0
            more = True
            
        
def main(searchEngine = "dual_encoders"):
    print("BOT: Hello, I am a TaskBot and I can help you with cooking tasks. Which recipe would you like to do? \n You can also search recipes with serving sizes. (e.g. how to make chicken parmesan 4)")
    items_shown = 0
    former_searches = []
    
    while True:
        x = input(" USER: ")
        if x == "quit":
            print("BOT: Quitting...")
            break
        elif x == "back":
            if len(former_searches) <= 1:
                print("BOT: No more searches to return to.")
            else:
                former_searches.pop()
                result = search_recipe(former_searches[-1][0],former_searches[-1][1],former_searches[-1][2])
                if result == "Fail":
                    print("BOT: Search failed.")
                    print("BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? \n You can also search recipes with serving sizes.")
                elif result == "back":
                    print("BOT: Going back to searching.")
                    print("BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? \n You can also search recipes with serving sizes.")
                    pass
                else:
                    print_recipe(result)
                
        elif x == "help":
            print("BOT: Commands: \n start - starts the application \n quit - quits the aplication \n any other input- searches a recipe")
        else:
            withI = input("BOT: What ingredients would you like to include? Input None if no need to specify.")
            withoutI = input("BOT: What ingredients would you like to exclude? Input None if no need to specify.")
            
            former_searches.append([x, withI, withoutI])
            result = search_recipe(x,withI,withoutI)
            if result == "Fail":
                print("BOT: Search failed.")
                print("BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? \n You can also search recipes with serving sizes.")
            elif result == "back":
                print("BOT: Going back to searching.")
                print("BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? \n You can also search recipes with serving sizes.")
                pass
            else:
                print("BOT: Here is the full recipe.")
                print_recipe(result)
                start_recipe(result)
                
    print("End")

#uncomment when changing original dataset or index structure or restarting kernel
initialize()
init.open_recipe_book()
main()
    



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user201'}
BOT: Hello, I am a TaskBot and I can help you with cooking tasks. Which recipe would you like to do? 
 You can also search recipes with serving sizes. (e.g. how to make chicken parmesan 4)


 USER:  how to make chicken parmesan 4
BOT: What ingredients would you like to include? Input None if no need to specify. 
BOT: What ingredients would you like to exclude? Input None if no need to specify. 


BOT: Great! These are the results I found:


BOT: Select a number from 1 to 4, type 'more' to show more results or 'back' go back to searching:  more


BOT: Great! These are the results I found:


...

BOT: Going back to searching.
BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? 
 You can also search recipes with serving sizes.


 USER:  parmasan
BOT: What ingredients would you like to include? Input None if no need to specify. chicken
BOT: What ingredients would you like to exclude? Input None if no need to specify. beef


BOT: No relevant results found.
BOT: Search failed.
BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? 
 You can also search recipes with serving sizes.


 USER:  parmesan
BOT: What ingredients would you like to include? Input None if no need to specify. chicken
BOT: What ingredients would you like to exclude? Input None if no need to specify. beef


BOT: No relevant results found.
BOT: Search failed.
BOT: Seems you didn't find what you were looking for. Which recipe would you like to do? 
 You can also search recipes with serving sizes.


In [ ]:
arr = list(range(1, 10))
print(arr)
b = 1
for i in arr[0:5]:
    print (i)
    b+=1
    

In [ ]:
for i in init.recipe_book.values():
    if "brownies" in i['displayName'].lower(): print(i)